# Lambda 1: Extracting Entities for Form Enrollment

In [54]:
import boto3
import json
import os
import sys
import base64
import time
import tzlocal
import re
from botocore.exceptions import ClientError
import logging

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

#Defining the bucket and file names.
bucket = 'ch-agent-assist-prompt-library-bucket'
file_key = 'prompts_library.json' 

##Processing data to fetch role and content
def data_preprocessing(data):
    #data = json_data['Records'][0]["body"]
    convo = ""
    content = json.loads(data["body"])["body"]["transcript"][0]["Content"]
    role = json.loads(data["body"])["body"]["transcript"][0]["ParticipantRole"]
    convo = convo + role + " : " + content
    
    return convo

#Defining LLM function for the prompt generator for entity extraction  
entities = "name of patient, status of insurance, insurance number, demographic details etc."
def get_prompt(bucket,file,prompt_category,required_prompt,conversation):
    entities = "name of patient, status of insurance, insurance number, demographic details etc."
    s3 = boto3.client('s3') 
    response = s3.get_object(Bucket=bucket,Key=file)
    content = response['Body'].read().decode('utf-8')
    json_content = json.loads(content)
    prompt = json_content[prompt_category][required_prompt].format(conversation=conversation,entities=entities)
    
    return prompt


    return json_response 

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]

    #end_index = end_char_indices[len(end_char_indices)-1]
    
    if len(end_char_indices) == 0:
        data2 = data1 + '}'
    else:
        data2 = data1
    
    end_char_indices_1 = [i.start() for i in re.finditer("}",data2)]
    print(f'end_indices:{end_char_indices_1}')  
    end_index = end_char_indices_1[len(end_char_indices_1)-1]
    print(f'end_index:{end_index}')
    
    if end_index == len(data2)-1:
        result = data2[start_index_final:]
    else:
        result = data2[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result

def SQS_Publisher(json_data):
    try:
        sqs = boto3.client('sqs',region_name='us-east-1')
        # Define the SQS queue URL
        queue_url = 'https://sqs.us-east-1.amazonaws.com/383299343633/ch-agent-assist-sqs'
        response = sqs.send_message(QueueUrl= queue_url,MessageBody=json.dumps(json_data) )
    except Exception as e:
        print(e)
        
def SQS_data_postprocessing(event,json_data):
    json_response = {
            "stream": "FORM_DATA",
            "streamConnectionId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
             "body": {
                 "transactionId": "f830e890-3ff2-4fdc-a08e-dd9b78a2dc28",
                  "contactId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
            "form_data": json_data,
                     }  
            }
###Below is the LLM model to extract entities
#Defining function to connect to Bedrock LLM
def load_titan_text_exp_v1(bedrock_runtime , prompt , temp , top_p):
    try:
        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 4096,
                "stopSequences": [],
                "temperature": 0,
                "topP": 1
            }
        })

        response = bedrock_runtime.invoke_model(
            body=body, modelId="amazon.titan-text-express-v1",accept="application/json", contentType="application/json"
        )
        
        response_body = json.loads(response.get("body").read())

        completion = ""
        for result in response_body['results']:
            completion += "\n" + result['outputText']

        return completion

    except ClientError:
        logging.error("Couldn't invoke Titan Text Express v1")
        raise

# Lambda handler to intgerate with AWS
def lambda_handler(event,context):
    start_time = time.time()
    final_transcript = ""
    for i in range(len(event["Records"])):
        final_transcript += "\n" + data_preprocessing(event["Records"][i])
        
    end_time1 = time.time()    # Record the end time

    # Calculate the elapsed time and log it
    elapsed_time1 = end_time1 - start_time
    print(f"Elapsed Time after calling data_preprocessing: {elapsed_time1} seconds")
    
    prompt_enrollment = get_prompt(bucket,file_key,'entity_extraction','enrollment_form_titan',final_transcript)
    end_time2 = time.time()
    elapsed_time2 = end_time2 - end_time1
    print(f"Elapsed Time after acessing_prompt_library_from_s3_bucket_to_generate_prompt: {elapsed_time2} seconds")
    
    enrollment_data = load_titan_text_exp_v1(bedrock_runtime,prompt_enrollment,0,0.9)
    end_time3 = time.time()
    elapsed_time3 = end_time3 - end_time2
    print(f"Elapsed Time after model calling: {elapsed_time3} seconds")
    
    #json_data = data_postprocessing(enrollment_data)
    #end_time4 = time.time()
    #elapsed_time4 = end_time4 - end_time3
    #print(f"Elapsed Time after model output postprocessing: {elapsed_time4} seconds")
    
    #enrollment_json_object = json.loads(json_data)
    #end_time5 = time.time()
    #elapsed_time5= end_time5 - end_time4
    #print(f"Elapsed Time after converting output data to json: {elapsed_time5} seconds")
    
    #print(event["Records"][0]['body'])
    #print(json.loads(event["Records"][0]['body'])["streamConnectionId"])
    #sqs_data = SQS_data_postprocessing(event,enrollment_json_object)
    #end_time6 = time.time()
    #elapsed_time6 = end_time6 - end_time5
    #print(f"Elapsed Time after sqs data postprocessing: {elapsed_time6} seconds")
    
    #SQS_Publisher(sqs_data)
    #end_time7 = time.time()
    #elapsed_time7 = end_time7 - end_time6
    #print(f"Elapsed Time after sending data to SQS: {elapsed_time7} seconds")
    return enrollment_data#{"statusCode": 200,"body": "Data Sent to SQS Sucessfully"}

In [55]:
data = {
  "Records": [
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"CUSTOMER\",\"ParticipantRole\":\"CUSTOMER\",\"Content\":\"Sure,my insurance is with Anthem INC.\",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    },
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"AGENT\",\"ParticipantRole\":\"AGENT\",\"Content\":\"Thank you,can you share your policy number? \",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    },
  ]
}

# Testing

In [62]:
lambda_handler(data,"")

Elapsed Time after calling data_preprocessing: 6.532669067382812e-05 seconds
Elapsed Time after acessing_prompt_library_from_s3_bucket_to_generate_prompt: 0.1562364101409912 seconds
Elapsed Time after model calling: 2.046919584274292 seconds


'\n\n```\n{\n    "rows": [\n        {\n            "Name of the Patient": "CUSTOMER",\n            "Status of Insurance": "Subscriber",\n            "Insurance Number": "{CA_HEALTH_NUMBER}",\n            "Demographic Details": "Age: 30, Gender: Male"\n        }\n    ]\n}\n```'

In [63]:
print('\n\n```\n{\n    "rows": [\n        {\n            "Name of the Patient": "CUSTOMER",\n            "Status of Insurance": "Subscriber",\n            "Insurance Number": "{CA_HEALTH_NUMBER}",\n            "Demographic Details": "Age: 30, Gender: Male"\n        }\n    ]\n}\n```')



```
{
    "rows": [
        {
            "Name of the Patient": "CUSTOMER",
            "Status of Insurance": "Subscriber",
            "Insurance Number": "{CA_HEALTH_NUMBER}",
            "Demographic Details": "Age: 30, Gender: Male"
        }
    ]
}
```


# Load Model Testing

In [41]:
sample_prompt = "Sample Prompt"
#Defining function to connect to Bedrock LLM
def load_titan_text_exp_v1(bedrock_runtime , prompt , temp , top_p):
    try:
        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 4096,
                "stopSequences": [],
                "temperature": 0,
                "topP": 1
            }
        })

        response = bedrock_runtime.invoke_model(
            body=body, modelId="amazon.titan-text-express-v1",accept="application/json", contentType="application/json"
        )
        
        response_body = json.loads(response.get("body").read())

        completion = ""
        for result in response_body['results']:
            completion += "\n" + result['outputText']

        return completion

    except ClientError:
        logging.error("Couldn't invoke Titan Text Express v1")
        raise
load_titan_text_exp_v1(bedrock_runtime , sample_prompt , 0 , 1)

'\n\nHello! I am a computer program designed to assist you. How can I help you today?'

In [40]:
t

{'inputTextTokenCount': 2,
 'results': [{'tokenCount': 20,
   'outputText': '\nHello! I am a computer program designed to assist you. How can I help you today?',
   'completionReason': 'FINISH'}]}

In [33]:
t = load_titan_text_exp_v1(bedrock_runtime , sample_prompt , 0 , 1)

In [39]:
t['results'][0]['outputText']

'\nHello! I am a computer program designed to assist you. How can I help you today?'